In [ ]:
import json
import gradio as gr

from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import FastEmbedEmbeddings
from langchain.vectorstores.utils import filter_complex_metadata
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

from langchain_community.embeddings.ollama import OllamaEmbeddings


# Create the prompte from the template.
promptTemplate = """Answer the question as precise as possible using the provided context. If the answer is
    not contained in the context, say "answer not available in context" \n\n
    Context: {context}
    Question: {question}
    Answer:

     """
modelSel = ""


# Load the PDF file to ChromaDB
def loadDataFromPDFFile(filePath):
    loader = PyPDFLoader(filePath)
    pages = loader.load_and_split()
    chunks = filter_complex_metadata(pages)
    embedding = OllamaEmbeddings(model="nomic-embed-text")

    vector_store = Chroma.from_documents(documents=chunks, embedding=embedding)
    return vector_store


def modelResponse(message, history):
    llm = ChatOllama(model="llama3:8b-instruct-q6_K")

    prompt = PromptTemplate(
        template=promptTemplate, input_variables=["context", "question"]
    )

    # Initiate the retriever
    dbLoaded = loadDataFromPDFFile("../data/HP_Book1_Chapter_Excerpt.pdf")
    retriever = dbLoaded.as_retriever(
        search_type="similarity_score_threshold",
        search_kwargs={"k": 5, "score_threshold": 0.2},
    )
    chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    return chain.invoke(message)


if __name__ == "__main__":

    # read configuration file
    # conf = {}
    # with open("config.json", "r") as confFile:
    #     conf = json.load(confFile)
    #     print(conf["model"])

    chatUI = gr.ChatInterface(fn=modelResponse, title="Harry Potter Story Q&A")
    chatUI.launch()

In [ ]:
# WebBaseLoader

from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(
    web_paths=["https://wix-doc.com/cv/"],
    # bs_kwargs=dict(
    #     parse_only=bs4.SoupStrainer(
    #         class_=("post-content", "post-title", "post-header")
    #     )
    # ),
)
docs = loader.load()
docs

In [5]:
# PyPDFLoader

from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("../data/cv-resume-wickeswong-2024-04-21.pdf")
docs = loader.load_and_split()
docs

[Document(page_content='Wickes Wong\nlinkedin.com/in/wickes-w  | wickeswong@gmail.com  | wix-doc.com | github.com/wickes1\nSkills\nDevelopment: Python, Go, JavaScript, TypeScript, RESTful API, Git, Shell Scripting, FastAPI, Next.js, React\nData and Analytics:  Pandas, PySpark, Airflow, Kafka, SQL, Grafana, Tableau, PostgreSQL, LangChain\nInfrastructure: AWS (S3, Lambda, EKS, ECS, Route53, RDS), Docker, Kubernetes, Ansible, Terraform, Helm, Kustomize, \nGitHub Actions, GitLab CI/CD, ArgoCD\nExperience\nData Engineer, Varadise Limited June 2021 – May 2023\n\uf06cLed the implementation of an event-driven message distribution system with Kafka and AWS EKS, enabling real-\ntime and cross-channel event notifications. Achieved sub-second message consumption and production (under \n100 milliseconds) for high-performance event delivery. Reduced customization time for output channels and \ntemplates from weeks to minutes through Go Templates.\n\uf06cImplemented scalable ETL pipelines with Python

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.ollama import OllamaEmbeddings


text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=0)
splits = text_splitter.split_documents(docs)
embedding = OllamaEmbeddings(model="nomic-embed-text")
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding)
retriever = vectorstore.as_retriever(
    # search_type="similarity_score_threshold",
    # search_kwargs={"k": 5, "score_threshold": 0.2},


)
splits

[Document(page_content='Wickes Wong\nlinkedin.com/in/wickes-w  | wickeswong@gmail.com  | wix-doc.com | github.com/wickes1', metadata={'source': '../data/cv-resume-wickeswong-2024-04-21.pdf', 'page': 0}),
 Document(page_content='Skills', metadata={'source': '../data/cv-resume-wickeswong-2024-04-21.pdf', 'page': 0}),
 Document(page_content='Development: Python, Go, JavaScript, TypeScript, RESTful API, Git, Shell Scripting, FastAPI,', metadata={'source': '../data/cv-resume-wickeswong-2024-04-21.pdf', 'page': 0}),
 Document(page_content='Next.js, React', metadata={'source': '../data/cv-resume-wickeswong-2024-04-21.pdf', 'page': 0}),
 Document(page_content='Data and Analytics:  Pandas, PySpark, Airflow, Kafka, SQL, Grafana, Tableau, PostgreSQL, LangChain', metadata={'source': '../data/cv-resume-wickeswong-2024-04-21.pdf', 'page': 0}),
 Document(page_content='Infrastructure: AWS (S3, Lambda, EKS, ECS, Route53, RDS), Docker, Kubernetes, Ansible, Terraform,', metadata={'source': '../data/cv-re

In [13]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama

promptTemplate = """Answer the question as precise as possible using the provided context. If the answer is not contained in the context, say "answer not available in context"
    Context: {context}
    Question: {question}
    Answer:
     """

llm = ChatOllama(model="llama3:8b-instruct-q6_K")
prompt = PromptTemplate(
    template=promptTemplate, input_variables=["context", "question"]
)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [14]:
example_messages = prompt.invoke(
    {"context": retriever | format_docs, "question": "filler question"}
).to_messages()
example_messages

[HumanMessage(content='Answer the question as precise as possible using the provided context. If the answer is not contained in the context, say "answer not available in context"\n    Context: first=VectorStoreRetriever(tags=[\'Chroma\', \'OllamaEmbeddings\'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7a752ab10140>) last=RunnableLambda(format_docs)\n    Question: filler question\n    Answer:\n     ')]

In [ ]:
question = "Tell me something about the candidate"

In [ ]:
answer = chain.invoke(question)
print(answer)

In [ ]:
vectorstore.delete_collection()

In [ ]:
retrieved_docs = retriever.invoke(question)
retrieved_docs

In [ ]:
import gradio as gr

def greet(name):
    return "Hello " + name + "!"

demo = gr.Interface(fn=greet, inputs="textbox", outputs="textbox")

demo.launch(share=True)  # Share your demo with just 1 extra parameter 🚀

In [8]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [9]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [11]:
from langchain_core.messages import HumanMessage

chat_history = []

question = "Who is Wickes Wong?"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=question), ai_msg_1["answer"]])

second_question = "Shall I hire him if I am a recruiter?"
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_2["answer"])

I don't have enough information to make a recommendation on hiring Wickes Wong. The provided context only shows his professional experience and skills as a Data Engineer, but it's limited in scope and doesn't provide a comprehensive view of his qualifications or fit for your specific needs. It would be best to conduct a more thorough evaluation and assessment before making a hiring decision.


In [12]:
for document in ai_msg_2["context"]:
    print(document)
    print()

page_content='Wickes Wong\nlinkedin.com/in/wickes-w  | wickeswong@gmail.com  | wix-doc.com | github.com/wickes1' metadata={'page': 0, 'source': '../data/cv-resume-wickeswong-2024-04-21.pdf'}

page_content='Data Engineer, Varadise Limited June 2021 – May 2023' metadata={'page': 0, 'source': '../data/cv-resume-wickeswong-2024-04-21.pdf'}

page_content='\uf06cOptimized prompts for accurate addressing of syllabus-related inquiries.' metadata={'page': 0, 'source': '../data/cv-resume-wickeswong-2024-04-21.pdf'}

page_content='Data Infrastructures (SDIs) into PostGIS and GeoServer. Enabled OGC Web Services for frontend' metadata={'page': 0, 'source': '../data/cv-resume-wickeswong-2024-04-21.pdf'}

